In [2]:
import os
import random
import collections
import operator
import itertools
import pickle
import tqdm

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import simpleclock
import sklearn.metrics

In [4]:
import torch
import torchtext

## Dataset, iterators

In [5]:
data_path = os.path.join(os.path.abspath(''), "data_cine_cleaned.csv")

In [6]:
TEXT = torchtext.data.Field(tokenize = "spacy",
                            tokenizer_language="fr_core_news_sm",
                            include_lengths=True)
LABEL = torchtext.data.LabelField(dtype=torch.float, use_vocab=False, preprocessing=lambda x: float(x) / 5)
# labels are linearly rescaled to a 0-1 range
# todo: test if preprocessing data before isn't faster

In [7]:
dataset = torchtext.data.TabularDataset(path=data_path,
                                        format="CSV",
                                        fields={"critique": ("input", TEXT), "note": ("target", LABEL)})

In [8]:
data_train, data_test = dataset.split()

In [9]:
data_train, data_valid = data_train.split()

In [10]:
print(f"""training data: {len(data_train)} examples.
validation data: {len(data_valid)} examples.
test data: {len(data_test)} examples.""")

training data: 36057 examples.
validation data: 15453 examples.
test data: 22076 examples.


In [11]:
vectors = torchtext.vocab.Vectors("cc.fr.300.vec", os.path.join(os.path.expanduser("~"), "Downloads"))

In [12]:
VOCAB_MAX_SIZE = 50000
TEXT.build_vocab(data_train, max_size=VOCAB_MAX_SIZE, vectors=vectors)

In [13]:
DEVICE = torch.device("cuda")

In [14]:
BATCH_SIZE = 256

In [15]:
iter_train, iter_valid, iter_test = \
    torchtext.data.BucketIterator.splits(datasets=(data_train, data_valid, data_test),
                                         batch_size=BATCH_SIZE,
                                         device=DEVICE,
                                         sort_within_batch=True,
                                         sort_key=lambda example: len(example.input),
                                         sort=False)

## Model definition

In [16]:
class RNN(torch.nn.Module):
    def __init__(self, n_vocab, embedding_dim, hidden_dim, output_dim, dropout, bidirectional,
                 n_layers, pad_idx):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.bidirectional = bidirectional
        num_dir = 2 if bidirectional else 1
        self.embedding = torch.nn.Embedding(n_vocab, embedding_dim, padding_idx=pad_idx)
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim,
                                 bidirectional=bidirectional,
                                 num_layers=n_layers)
        self.fc = torch.nn.Linear(hidden_dim * num_dir, output_dim)
        self.dropout = torch.nn.Dropout(dropout)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, input_lengths):
        input, lengths = input_lengths
        torch.nn.utils.rnn.pack_padded_sequence(input, lengths)
        embedded = self.embedding(input)  # ((sent_len, batch), emb_dim)
        packed_output, (hidden, cell) = self.rnn(embedded)  # hidden: (num_layers * num_directions,
                                                            #          batch, hidden_size * num_directions)
        hidden = (torch.cat([hidden[-2, :, :], hidden[-1, :, :]], dim=1)
                  if self.bidirectional else hidden).squeeze(0)  # (batch, hidden_size * num_directions)
        return self.sigmoid(self.fc(self.dropout(hidden)))  # (batch, 1)

In [17]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

DEFAULT_PARAMS = {
    "n_vocab": len(TEXT.vocab),
    "embedding_dim": 300,
    "hidden_dim": 256,
    "output_dim": 1,
    "dropout": 0.5,
    "bidirectional": True,
    "n_layers": 1,
    "pad_idx": PAD_IDX,
}

def default_model(**kwargs):
    _d = {}
    _d.update(DEFAULT_PARAMS)
    _d.update(kwargs)
    return RNN(**_d)

In [18]:
def pseudo_init(model, criterion, device=DEVICE, learn_embedding_param=True):
    model.embedding.weight.data.copy_(TEXT.vocab.vectors)
    model.embedding.weight.data[UNK_IDX] = torch.zeros(model.embedding_dim)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(model.embedding_dim)
    
    for name, param in model.named_parameters():
        if name == "embedding.weight":
            param.requires_grad = learn_embedding_param
    
    print("The model has {:,} trainable parameters"
         .format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
    
    model = model.to(device)
    criterion = criterion.to(device)
    
    return model, criterion

In [19]:
def output_to_pred(output):
    return (output * 10).round() / 2

## Training utils

In [20]:
def accuracy(preds, y):
    correct = (preds == y).float()  # convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [21]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        output = model(batch.input).squeeze(1)
        loss = criterion(output, batch.target)
        loss.backward()
        optimizer.step()
        
        acc = accuracy(output_to_pred(output), batch.target * 5)
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
      
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            output = model(batch.input).squeeze(1)
            loss = criterion(output, batch.target)
            acc = accuracy(output_to_pred(output), batch.target * 5)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
class TrainInfo:
    def __init__(self, valid={}, train={}):
        self.valid = collections.defaultdict(lambda: [])
        self.valid.update(valid)
        self.train = collections.defaultdict(lambda: [])
        self.train.update(train)
    
    def save(self, path):
        packed = {
            "valid": dict(self.valid),
            "train": dict(self.train),
        }
        with open(path, "wb") as f:
            pickle.dump(packed, f)
    
    @classmethod
    def load(cls, path):
        with open(path, "rb") as f:
            packed = pickle.load(f)
            return cls(valid=packed["valid"],
                       train=packed["train"])
    
    @staticmethod
    def _dict_to_repr(d):
        return dict(map(lambda k_v: (k_v[0], f"{len(k_v[1])} elements"), d.items()))
    
    def __repr__(self):
        return pprint.pformat({"valid": self._dict_to_repr(self.valid),
                     "train": self._dict_to_repr(self.train),})

In [24]:
def do_training(model, name, iter_train, iter_valid, optimizer, criterion, fun_train,
                fun_eval, n_epochs=100, train_info=None):

    clock = simpleclock.Clock.started()
    torch.cuda.empty_cache()
    train_info = train_info if train_info is not None else TrainInfo()
    best_valid_loss = min(train_info.valid["loss"]) if train_info.valid["loss"] else float("inf")
    
    for epoch in range(n_epochs):

        clock.elapsed_since_start.call()  # meh

        train_loss, train_acc = fun_train(model, iter_train, optimizer, criterion)
        valid_loss, valid_acc = fun_eval(model, iter_valid, criterion)
        
        is_best = valid_loss < best_valid_loss
        print("Epoch: {e:<3}. T, V acc: {train:.1f}%, {valid:.1f}%. Took {t:.2}s."
             .format(e=epoch + 1,
                     train=100 * train_acc,
                     valid=100 * valid_acc,
                     t=clock.elapsed_since_last_call())
             + (" (+)" if is_best else "")
             )
        
        train_info.train["loss"].append(train_loss)
        train_info.valid["loss"].append(valid_loss)

        if is_best:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), f"{name}.pt")

    clock.elapsed_since_start.print(f"Trained {name}, {n_epochs} epochs, for")
    return train_info

In [25]:
class TrainSet:
    def __init__(self, model, name, iter_train, iter_valid,
                 fun_optimizer, fun_criterion, fun_train, fun_eval,
                 device=DEVICE, n_epochs=100):
        self.model = model
        self.name = name
        self.iter_train = iter_train
        self.iter_valid = iter_valid
        self.fun_optimizer = fun_optimizer
        self.fun_criterion = fun_criterion
        self.fun_train = fun_train
        self.fun_eval = fun_eval
        self.n_epochs = n_epochs
        self.device = device
        
        self.optimizer = None
        self.criterion = None
        
    def init(self, learn_embedding_param=True):
        self.model, self.criterion = pseudo_init(self.model, self.fun_criterion(), self.device,
                                                 learn_embedding_param=learn_embedding_param)
        self.optimizer = self.fun_optimizer(self.model.parameters())
    
    def do_training(self):
        if self.optimizer is None or self.criterion is None:
            raise Exception("It looks like an init is needed: optimizer or criterion is None")
        return do_training(model=self.model,
                           name=self.name,
                           iter_train=self.iter_train,
                           iter_valid=self.iter_valid,
                           optimizer=self.optimizer,
                           criterion=self.criterion,
                           fun_train=self.fun_train,
                           fun_eval=self.fun_eval,
                           n_epochs=self.n_epochs)

## Saving/loading utils

In [26]:
def save_vocab_embedding(path, vocab, embedding):
    with open(path, "w") as f:
        for word, vector in tqdm.tqdm(zip(vocab.itos, embedding)):
            
            # skip words with unicode symbols
            if len(word) != len(word.encode()):
                continue
            
            # 'words' like " " or "\n" fail to be loaded
            if word.strip() == "":
                continue

            f.write(f"{word} {' '.join(str(e) for e in vector.tolist())}\n")

In [27]:
def load_context(model, field, path_model, path_vocab, cache_embeddings="cache_embeddings", device=DEVICE):
    
    _vectors = torchtext.vocab.Vectors(path_vocab, cache_embeddings)  # voir unk_init
    field.build_vocab(data_train, max_size=VOCAB_MAX_SIZE, vectors=_vectors)
    
    model.load_state_dict(torch.load(path_model))
    
    model.embedding.weight.data.copy_(field.vocab.vectors)
    
    model = model.to(device)
    
    iter_train, iter_valid, iter_test = \
        torchtext.data.BucketIterator.splits(datasets=(data_train, data_valid, data_test),
                                             batch_size=BATCH_SIZE,
                                             device=DEVICE,
                                             sort_within_batch=True,
                                             sort_key=lambda example: len(example.input),
                                             sort=False)
    
    return model, (iter_train, iter_valid, iter_test)

## Training

In [28]:
train_sets = []
N_EPOCHS = 10000

for hidden_dim, n_layers in itertools.product([256], [3]):
    train_sets.append(TrainSet(
        model=default_model(hidden_dim=hidden_dim, n_layers=n_layers),
        name=f"rnn_hidden-{hidden_dim}_nlayers-{n_layers}_nepochs-{N_EPOCHS}",
        iter_train=iter_train,
        iter_valid=iter_valid,
        fun_optimizer=torch.optim.Adam,
        fun_criterion=torch.nn.MSELoss,
        fun_train=train,
        fun_eval=evaluate,
        n_epochs=N_EPOCHS
    ))

In [29]:
for train_set in train_sets:
    train_set.init()
    train_info = train_set.do_training()
    
    train_info.save(f"info_{train_set.name}.pickle")
    
    # plot loss data
    fig, ax = plt.subplots(figsize=(20, 8))
    ax_train = ax.plot(list(range(train_set.n_epochs)), train_info.train["loss"], label="train")
    ax_valid = ax.plot(list(range(train_set.n_epochs)), train_info.valid["loss"], label="valid")
    fig.legend()
    fig.suptitle("Validation loss")
    plt.show()
    
    # save
    save_vocab_embedding("vocab_emb", TEXT.vocab, model.embedding.weight)

The model has 19,297,817 trainable parameters
Epoch: 1  . T, V acc: 20.1%, 27.1%. Took: 11.32s
Epoch: 2  . T, V acc: 29.9%, 27.5%. Took: 11.22s
Epoch: 3  . T, V acc: 32.0%, 24.5%. Took: 11.05s
Epoch: 4  . T, V acc: 37.4%, 23.2%. Took: 11.07s
Epoch: 5  . T, V acc: 44.3%, 26.2%. Took: 11.22s
Epoch: 6  . T, V acc: 50.7%, 24.6%. Took: 11.49s
Epoch: 7  . T, V acc: 59.3%, 25.7%. Took: 11.24s
Epoch: 8  . T, V acc: 64.7%, 24.8%. Took: 11.29s
Epoch: 9  . T, V acc: 69.6%, 25.2%. Took: 11.12s
Epoch: 10 . T, V acc: 72.0%, 25.8%. Took: 11.03s
Epoch: 11 . T, V acc: 77.1%, 26.6%. Took: 11.02s
Epoch: 12 . T, V acc: 80.7%, 26.8%. Took: 11.23s
Epoch: 13 . T, V acc: 83.8%, 27.7%. Took: 11.20s
Epoch: 14 . T, V acc: 85.2%, 27.8%. Took: 11.18s
Epoch: 15 . T, V acc: 85.5%, 28.0%. Took: 11.14s
Epoch: 16 . T, V acc: 87.1%, 28.9%. Took: 11.11s
Epoch: 17 . T, V acc: 85.8%, 29.6%. Took: 11.16s
Epoch: 18 . T, V acc: 86.1%, 28.0%. Took: 11.17s
Epoch: 19 . T, V acc: 88.7%, 29.9%. Took: 11.16s
Epoch: 20 . T, V acc: 9

Epoch: 168. T, V acc: 99.7%, 41.4%. Took: 11.31s
Epoch: 169. T, V acc: 99.8%, 41.5%. Took: 11.37s
Epoch: 170. T, V acc: 99.6%, 39.2%. Took: 11.29s
Epoch: 171. T, V acc: 99.6%, 40.5%. Took: 11.21s
Epoch: 172. T, V acc: 99.1%, 40.6%. Took: 11.28s
Epoch: 173. T, V acc: 99.4%, 40.8%. Took: 11.26s
Epoch: 174. T, V acc: 99.6%, 40.2%. Took: 11.27s
Epoch: 175. T, V acc: 99.7%, 40.8%. Took: 11.29s
Epoch: 176. T, V acc: 99.7%, 40.6%. Took: 11.29s
Epoch: 177. T, V acc: 99.7%, 40.7%. Took: 11.31s
Epoch: 178. T, V acc: 99.8%, 40.6%. Took: 11.28s
Epoch: 179. T, V acc: 99.8%, 40.3%. Took: 11.27s
Epoch: 180. T, V acc: 99.7%, 40.6%. Took: 11.30s
Epoch: 181. T, V acc: 99.8%, 41.0%. Took: 11.30s
Epoch: 182. T, V acc: 99.7%, 40.3%. Took: 11.30s
Epoch: 183. T, V acc: 99.8%, 40.7%. Took: 11.31s
Epoch: 184. T, V acc: 99.8%, 41.1%. Took: 11.33s
Epoch: 185. T, V acc: 99.8%, 40.7%. Took: 11.28s
Epoch: 186. T, V acc: 99.8%, 41.1%. Took: 11.32s
Epoch: 187. T, V acc: 99.8%, 40.7%. Took: 11.30s
Epoch: 188. T, V acc

Epoch: 336. T, V acc: 99.8%, 41.1%. Took: 11.30s
Epoch: 337. T, V acc: 99.8%, 41.1%. Took: 11.27s
Epoch: 338. T, V acc: 99.8%, 41.2%. Took: 11.23s
Epoch: 339. T, V acc: 99.8%, 41.2%. Took: 11.24s
Epoch: 340. T, V acc: 99.8%, 41.2%. Took: 11.26s
Epoch: 341. T, V acc: 99.8%, 41.4%. Took: 11.24s
Epoch: 342. T, V acc: 99.8%, 41.5%. Took: 11.25s
Epoch: 343. T, V acc: 99.8%, 41.6%. Took: 11.28s
Epoch: 344. T, V acc: 99.8%, 40.8%. Took: 11.26s
Epoch: 345. T, V acc: 99.8%, 40.6%. Took: 11.27s
Epoch: 346. T, V acc: 99.8%, 40.9%. Took: 11.29s
Epoch: 347. T, V acc: 99.8%, 40.9%. Took: 11.28s
Epoch: 348. T, V acc: 99.6%, 40.6%. Took: 11.30s
Epoch: 349. T, V acc: 99.6%, 40.4%. Took: 11.36s
Epoch: 350. T, V acc: 99.7%, 40.6%. Took: 11.31s
Epoch: 351. T, V acc: 99.8%, 41.1%. Took: 11.31s
Epoch: 352. T, V acc: 99.8%, 40.9%. Took: 11.24s
Epoch: 353. T, V acc: 99.8%, 41.3%. Took: 11.27s
Epoch: 354. T, V acc: 99.8%, 41.1%. Took: 11.27s
Epoch: 355. T, V acc: 99.8%, 41.4%. Took: 11.27s
Epoch: 356. T, V acc

Epoch: 504. T, V acc: 99.8%, 41.0%. Took: 11.26s
Epoch: 505. T, V acc: 99.8%, 41.8%. Took: 11.23s
Epoch: 506. T, V acc: 99.8%, 41.6%. Took: 11.25s
Epoch: 507. T, V acc: 99.8%, 40.7%. Took: 11.25s
Epoch: 508. T, V acc: 99.8%, 41.0%. Took: 11.27s
Epoch: 509. T, V acc: 99.8%, 41.5%. Took: 11.25s
Epoch: 510. T, V acc: 99.8%, 40.9%. Took: 11.23s
Epoch: 511. T, V acc: 99.8%, 41.0%. Took: 11.25s
Epoch: 512. T, V acc: 99.8%, 41.3%. Took: 11.20s
Epoch: 513. T, V acc: 99.8%, 41.3%. Took: 11.11s
Epoch: 514. T, V acc: 99.8%, 41.0%. Took: 11.07s
Epoch: 515. T, V acc: 99.8%, 41.2%. Took: 11.10s
Epoch: 516. T, V acc: 99.8%, 41.3%. Took: 11.08s
Epoch: 517. T, V acc: 99.8%, 40.5%. Took: 11.10s
Epoch: 518. T, V acc: 99.8%, 41.5%. Took: 11.08s
Epoch: 519. T, V acc: 99.8%, 41.5%. Took: 11.09s
Epoch: 520. T, V acc: 99.8%, 41.3%. Took: 11.10s
Epoch: 521. T, V acc: 99.8%, 41.5%. Took: 11.11s
Epoch: 522. T, V acc: 99.8%, 40.9%. Took: 11.17s
Epoch: 523. T, V acc: 99.8%, 41.4%. Took: 11.24s
Epoch: 524. T, V acc

Epoch: 672. T, V acc: 99.8%, 41.4%. Took: 11.26s
Epoch: 673. T, V acc: 99.8%, 41.5%. Took: 11.26s
Epoch: 674. T, V acc: 99.8%, 40.8%. Took: 11.32s
Epoch: 675. T, V acc: 99.8%, 41.5%. Took: 11.32s
Epoch: 676. T, V acc: 99.8%, 41.1%. Took: 11.29s
Epoch: 677. T, V acc: 99.8%, 40.5%. Took: 11.27s
Epoch: 678. T, V acc: 99.8%, 40.5%. Took: 11.32s
Epoch: 679. T, V acc: 99.8%, 41.3%. Took: 11.29s
Epoch: 680. T, V acc: 99.8%, 41.1%. Took: 11.28s
Epoch: 681. T, V acc: 99.8%, 41.6%. Took: 11.32s
Epoch: 682. T, V acc: 99.8%, 40.6%. Took: 11.29s
Epoch: 683. T, V acc: 99.8%, 40.7%. Took: 11.30s
Epoch: 684. T, V acc: 99.8%, 41.1%. Took: 11.29s
Epoch: 685. T, V acc: 99.8%, 40.3%. Took: 11.26s
Epoch: 686. T, V acc: 99.8%, 41.0%. Took: 11.22s
Epoch: 687. T, V acc: 99.8%, 41.4%. Took: 11.22s
Epoch: 688. T, V acc: 99.8%, 41.0%. Took: 11.24s
Epoch: 689. T, V acc: 99.8%, 41.4%. Took: 11.22s
Epoch: 690. T, V acc: 99.8%, 41.3%. Took: 11.21s
Epoch: 691. T, V acc: 99.8%, 41.1%. Took: 11.17s
Epoch: 692. T, V acc

...

Epoch: 1008. T, V acc: 99.8%, 40.8%. Took: 11.22s
Epoch: 1009. T, V acc: 99.8%, 40.6%. Took: 11.12s
Epoch: 1010. T, V acc: 99.8%, 40.6%. Took: 11.29s
Epoch: 1011. T, V acc: 99.8%, 40.5%. Took: 11.33s
Epoch: 1012. T, V acc: 99.8%, 40.6%. Took: 11.35s
Epoch: 1013. T, V acc: 99.8%, 40.5%. Took: 11.30s
Epoch: 1014. T, V acc: 99.8%, 40.5%. Took: 11.31s
Epoch: 1015. T, V acc: 99.9%, 40.2%. Took: 11.26s
Epoch: 1016. T, V acc: 99.8%, 40.2%. Took: 11.28s
Epoch: 1017. T, V acc: 99.8%, 40.5%. Took: 11.22s
Epoch: 1018. T, V acc: 99.8%, 40.9%. Took: 11.22s
Epoch: 1019. T, V acc: 99.8%, 40.9%. Took: 11.21s
Epoch: 1020. T, V acc: 99.8%, 40.9%. Took: 11.23s
Epoch: 1021. T, V acc: 99.9%, 41.0%. Took: 11.19s
Epoch: 1022. T, V acc: 99.8%, 41.4%. Took: 11.20s
Epoch: 1023. T, V acc: 99.8%, 41.2%. Took: 11.18s
Epoch: 1024. T, V acc: 99.8%, 41.0%. Took: 11.20s
Epoch: 1025. T, V acc: 99.9%, 41.1%. Took: 11.23s
Epoch: 1026. T, V acc: 99.9%, 41.1%. Took: 11.23s
Epoch: 1027. T, V acc: 99.9%, 41.0%. Took: 11.23s


Epoch: 1172. T, V acc: 99.9%, 40.8%. Took: 11.29s
Epoch: 1173. T, V acc: 99.8%, 41.0%. Took: 11.29s
Epoch: 1174. T, V acc: 99.8%, 41.4%. Took: 11.22s
Epoch: 1175. T, V acc: 99.8%, 40.8%. Took: 11.26s
Epoch: 1176. T, V acc: 99.9%, 41.0%. Took: 11.21s
Epoch: 1177. T, V acc: 99.8%, 41.2%. Took: 11.18s
Epoch: 1178. T, V acc: 99.9%, 41.1%. Took: 11.18s
Epoch: 1179. T, V acc: 99.8%, 40.6%. Took: 11.18s
Epoch: 1180. T, V acc: 99.8%, 41.0%. Took: 11.18s
Epoch: 1181. T, V acc: 99.8%, 41.0%. Took: 11.19s
Epoch: 1182. T, V acc: 99.8%, 41.1%. Took: 11.26s
Epoch: 1183. T, V acc: 99.8%, 41.2%. Took: 11.24s
Epoch: 1184. T, V acc: 99.8%, 40.7%. Took: 11.28s
Epoch: 1185. T, V acc: 99.8%, 40.9%. Took: 11.37s
Epoch: 1186. T, V acc: 99.8%, 41.0%. Took: 11.32s
Epoch: 1187. T, V acc: 99.8%, 40.6%. Took: 11.27s
Epoch: 1188. T, V acc: 99.9%, 41.4%. Took: 11.25s
Epoch: 1189. T, V acc: 99.8%, 41.2%. Took: 11.23s
Epoch: 1190. T, V acc: 99.8%, 41.4%. Took: 11.28s
Epoch: 1191. T, V acc: 99.8%, 40.9%. Took: 11.25s


Epoch: 1336. T, V acc: 99.8%, 41.0%. Took: 11.32s
Epoch: 1337. T, V acc: 99.8%, 41.0%. Took: 11.27s
Epoch: 1338. T, V acc: 99.8%, 40.5%. Took: 11.29s
Epoch: 1339. T, V acc: 99.8%, 40.3%. Took: 11.27s
Epoch: 1340. T, V acc: 99.8%, 40.4%. Took: 11.28s
Epoch: 1341. T, V acc: 99.8%, 40.5%. Took: 11.27s
Epoch: 1342. T, V acc: 99.8%, 40.6%. Took: 11.28s
Epoch: 1343. T, V acc: 99.9%, 40.5%. Took: 11.28s
Epoch: 1344. T, V acc: 99.8%, 40.5%. Took: 11.31s
Epoch: 1345. T, V acc: 99.8%, 40.7%. Took: 11.28s
Epoch: 1346. T, V acc: 99.8%, 40.6%. Took: 11.28s
Epoch: 1347. T, V acc: 99.8%, 40.8%. Took: 11.28s
Epoch: 1348. T, V acc: 99.8%, 40.8%. Took: 11.32s
Epoch: 1349. T, V acc: 99.8%, 40.8%. Took: 11.32s
Epoch: 1350. T, V acc: 99.8%, 40.9%. Took: 11.29s
Epoch: 1351. T, V acc: 99.8%, 41.0%. Took: 11.31s
Epoch: 1352. T, V acc: 99.8%, 40.9%. Took: 11.27s
Epoch: 1353. T, V acc: 99.8%, 40.7%. Took: 11.28s
Epoch: 1354. T, V acc: 99.9%, 40.8%. Took: 11.31s
Epoch: 1355. T, V acc: 99.8%, 40.9%. Took: 11.27s


Epoch: 1500. T, V acc: 99.9%, 41.0%. Took: 11.32s
Epoch: 1501. T, V acc: 99.8%, 40.9%. Took: 11.33s
Epoch: 1502. T, V acc: 99.8%, 40.9%. Took: 11.72s
Epoch: 1503. T, V acc: 99.8%, 41.0%. Took: 11.32s
Epoch: 1504. T, V acc: 99.8%, 40.4%. Took: 11.66s
Epoch: 1505. T, V acc: 99.9%, 40.5%. Took: 11.44s
Epoch: 1506. T, V acc: 99.8%, 41.1%. Took: 11.39s
Epoch: 1507. T, V acc: 99.8%, 41.2%. Took: 11.43s
Epoch: 1508. T, V acc: 99.9%, 41.2%. Took: 11.48s
Epoch: 1509. T, V acc: 99.8%, 41.2%. Took: 11.47s
Epoch: 1510. T, V acc: 99.9%, 41.2%. Took: 11.27s
Epoch: 1511. T, V acc: 99.8%, 41.4%. Took: 11.58s
Epoch: 1512. T, V acc: 99.8%, 41.4%. Took: 11.55s


KeyboardInterrupt: 

## Peeking

In [30]:
import spacy

nlp = spacy.load('fr_core_news_sm')

In [31]:
def predict_tokens(tokens, model, device=DEVICE):
    model.eval()
    idxs = [TEXT.vocab.stoi[t] for t in tokens]
    inp = torch.LongTensor(idxs).reshape(-1, 1).to(device)
    output = output_to_pred(model((inp, torch.LongTensor([len(tokens)]))))
    return output.item()


def predict(sentence, model):
    return predict_tokens(list(map(str, nlp.tokenizer(sentence))), model)

In [45]:
model = default_model(hidden_dim=256, n_layers=3)

In [46]:
model.load_state_dict(torch.load('rnn_hidden-256_nlayers-3_nepochs-10000.pt'))

<All keys matched successfully>

In [47]:
model = model.to(DEVICE)

In [48]:
predict("Du temps perdu.", model), \
predict("Un très bon film, à voir avec toute la famille.", model)

(3.0, 4.0)

In [49]:
for example in random.sample(list(data_train), 3):
    tokens, note = example.input, float(example.target)
    print(" ".join(tokens))
    print(f"true) {note * 5} - {predict_tokens(tokens, model)} (pred")

Un peu à l' image de sa filmographie , Roger Michell semble tiraillé entre un désir sincère de donner corps aux aspérités de son drame humain et ses habitudes de faiseur discipliné .
true) 3.0 - 3.0 (pred
Une rareté : le reggae conté par ceux qui le font . A chacun son histoire , parfois dure , toujours touchante .
true) 4.0 - 4.0 (pred
Ici , le morceau de sparadrap dont , à l’ image du capitaine Haddock , Elia Suleiman ne parvient pas à se défaire , c’ est son propre pays , la Palestine . Il en résulte une absurde comédie de l’ absurde , d’ une magnifique et tendre mélancolie . Indispensable .
true) 4.0 - 3.5 (pred


## Looser accuracies for evaluation

Is the model really doing bad if it predicts a 4.5 instead of a 5 ? There are at least two ways to allow for forgivable divergence with the test data :
* decrease notation's granularity, e.g. tranform the marks into good/bad, or good/bad/neutral.
* consider a prediction correct if it belongs to a 'small' interval containing the true value.

### Good/Neutral/Bad prediction

In [50]:
def normalized_to_3_way(pred_tensor, bad_treshold=.375, good_treshold=.625):
    """np array with values: 0: bad, 1: neutral, 2: good"""
    return np.digitize(pred_tensor.cpu().detach().numpy(), [bad_treshold, good_treshold])
    
def eval_accuracy_3w(model, iterator):
    n_examples = 0
    n_success = 0
    for batch in iterator:
        predictions = model(batch.input).squeeze(1)
        n_examples += len(batch)
        n_success += sum(normalized_to_3_way(predictions) == normalized_to_3_way(batch.target))
    return n_success / n_examples

def classif_report_3w(model, iterator):
    def preds_and_trues_to_array(predictions, true_targets):
        return np.concatenate([normalized_to_3_way(true_targets).reshape(-1, 1),
                               normalized_to_3_way(predictions).reshape(-1, 1)], axis=1)
    
    array = None
    for batch in iterator:
        predictions = model(batch.input).squeeze(1)
        
        if array is None:
            array = preds_and_trues_to_array(predictions, batch.target)
        else:
            array = np.concatenate([array,
                                    preds_and_trues_to_array(predictions, batch.target)], axis=0)
    print(sklearn.metrics.classification_report(array[:, 0],
                                                array[:, 1],
                                                labels=[0, 1, 2],
                                                target_names=["bad", "neutral", "good"]))

In [51]:
iterator = iter_test

classif_report_3w(model, iterator)


              precision    recall  f1-score   support

         bad       0.36      0.22      0.28      1661
     neutral       0.65      0.60      0.63      9672
        good       0.72      0.82      0.77     10743

    accuracy                           0.68     22076
   macro avg       0.58      0.55      0.56     22076
weighted avg       0.67      0.68      0.67     22076



### Fuzzy accuracy

In [40]:
def eval_accuracy_fuzzy(model, iterator, fuzziness=.1):
    n_examples = 0.
    n_success = 0.
    for batch in iterator:
        predictions = model(batch.input).squeeze(1)
        n_examples += len(batch)
        n_success += sum(torch.abs(predictions - batch.target) <= fuzziness).item()
    return n_success / n_examples

In [56]:
eval_accuracy_fuzzy(model, iter_test, fuzziness=.2)

0.7549827867367277